## Querying Nexus knowledge graph using SPARQL

The goal of this notebook is to learn the basics of SPARQL. Only the READ part of SPARQL will be exposed.


## Prerequisites

This notebook assumes you've created a project within the AWS deployment of Nexus. If not follow the Blue Brain Nexus [Quick Start tutorial](https://bluebrain.github.io/nexus/docs/tutorial/getting-started/quick-start/index.html).

## Overview

You'll work through the following steps:

1. Create a sparql wrapper around your project's SparqlView
2. Explore and navigate data using the SPARQL query language


## Step 1: Create a sparql wrapper around your project's SparqlView

Every project in Blue Brain Nexus comes with a SparqlView enabling to navigate the data as a graph and to query it using the W3C SPARQL Language. The address of such SparqlView is https://nexus-sandbox.io/v1/views/tutorialnexus/\$PROJECTLABEL/graph/sparql for a project withe label \$PROJECTLABEL. The address of a SparqlView is also called a **SPARQL endpoint**.

In [0]:
#Configuration for the Nexus deployment
nexus_deployment = "https://nexus-sandbox.io/v1"

token = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICItSm9GOUNPdnZ0N1VoeWhKTUMtWWxURjZwaVJsWmdRS1JRa3M1c1BNS3h3In0.eyJqdGkiOiIwNzI2NTc5ZS1mOTM3LTRjYTgtOTdlZi0xOWE4MjMzOTU0NGMiLCJleHAiOjE1NDkxODM1NzEsIm5iZiI6MCwiaWF0IjoxNTQ4NTc4NzcxLCJpc3MiOiJodHRwczovL25leHVzLXNhbmRib3guaW8vYXV0aC9yZWFsbXMvZ2l0aHViIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImY0ZmY1ZWMyLTJjOGQtNDM1YS1hZWFmLWUyMDU3MTJmMDYzYSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im5leHVzLXdlYiIsIm5vbmNlIjoiMTIzNDU2IiwiYXV0aF90aW1lIjoxNTQ4NTc4NzcxLCJzZXNzaW9uX3N0YXRlIjoiY2M5NGVkMWItOWM1OC00NmE5LWI0OWEtMjhlYzgzOGM4YTI2IiwiYWNyIjoiMSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBlbWFpbCIsInN1YiI6ImFnYXJjaSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwibmFtZSI6IkFtYW5kYSBHYXJjaSIsInByZWZlcnJlZF91c2VybmFtZSI6ImFnYXJjaSIsImdpdmVuX25hbWUiOiJBbWFuZGEiLCJmYW1pbHlfbmFtZSI6IkdhcmNpIiwiZW1haWwiOiJyaWdvdmlydXRhQGdtYWlsLmNvbSJ9.g1V9CP3s9XXNVuAbia1L1qX8jLI4qSimTSMFOVuCX6xrcvUyZ-bPFuPQeLVkkjGQuS973wd538eS9B9J2JgZxsK3Dkzv8yWMD7dOohLnEz-DWqAC8d-Pei_bVlf6PReiTru2ggt6VoeaoO9qXDVYEUKL0T12YsEFsyM687FIUDlwz5cVqo-aVA1rP-HUCZxTzf2peRW476dw6iRSJsDkhx3HhZNsHzk_U9GXCW916-Wc9KyshNQvYcJF5nv4o-U6qgnUqWz72aw13b2W7xD6_WLvl_2FgY35jVC9kOzIdGaI-0oDLmeW5LvgYJYD9skLuup3riY1Dbsqt97sPd3mtA"

org ="amld"
project ="recommender"

headers = {}

In [2]:
#Let install sparqlwrapper which a python wrapper around sparql client
!pip install git+https://github.com/RDFLib/sparqlwrapper

  Cloning https://github.com/RDFLib/sparqlwrapper to /tmp/pip-req-build-1vhbb4_d
    100% |████████████████████████████████| 348kB 5.4MB/s 
    100% |████████████████████████████████| 51kB 8.6MB/s 
  Running setup.py bdist_wheel for SPARQLWrapper ... - \ | / done
  Stored in directory: /tmp/pip-ephem-wheel-cache-puxoxzg7/wheels/94/87/ff/bd3f8bccb74168f42b5ba997f9105e18a9cc2699ad9a5525a6
Successfully built SPARQLWrapper


In [0]:
# Utility functions to create sparql wrapper around a sparql endpoint

from SPARQLWrapper import SPARQLWrapper, JSON, POST, GET, POSTDIRECTLY, CSV
import requests



def create_sparql_client(sparql_endpoint, http_query_method=POST, result_format= JSON, token=None):
    sparql_client = SPARQLWrapper(sparql_endpoint)
    #sparql_client.addCustomHttpHeader("Content-Type", "application/sparql-query")
    if token:
        sparql_client.addCustomHttpHeader("Authorization","Bearer {}".format(token))
    sparql_client.setMethod(http_query_method)
    sparql_client.setReturnFormat(result_format)
    if http_query_method == POST:
        sparql_client.setRequestMethod(POSTDIRECTLY)
    
    return sparql_client

In [0]:
# Utility functions
import pandas as pd

pd.set_option('display.max_colwidth', -1)

# Convert SPARQL results into a Pandas data frame
def sparql2dataframe(json_sparql_results):
    cols = json_sparql_results['head']['vars']
    out = []
    for row in json_sparql_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)
    return pd.DataFrame(out, columns=cols)

# Send a query using a sparql wrapper 
def query_sparql(query, sparql_client):
    sparql_client.setQuery(query)
    

    result_object = sparql_client.query()
    if sparql_client.returnFormat == JSON:
        return result_object._convertJSON()
    return result_object.convert()

In [0]:
# Let create a sparql wrapper around the project sparql view
sparqlview_endpoint = nexus_deployment+"/views/"+org+"/"+project+"/graph/sparql"
sparqlview_wrapper = create_sparql_client(sparql_endpoint=sparqlview_endpoint, token=token,http_query_method= POST, result_format=JSON)

## Step 2: Explore and navigate data using the SPARQL query language


Let write our first query.

In [0]:
select_all_query = """
SELECT ?s ?p ?o
WHERE
{
  ?s ?p ?o
}
OFFSET 0
LIMIT 5
"""

nexus_results = query_sparql(select_all_query,sparqlview_wrapper)

nexus_df =sparql2dataframe(nexus_results)
nexus_df.head()

Most SPARQL queries you'll see will have the anotomy above with:
* a **SELECT** clause that let you select the variables you want to retrieve
* a **WHERE** clause defining a set of constraints that the variables should satisfy to be retrieved
* **LIMIT** and **OFFSET** clauses to enable pagination
* the constraints are usually graph patterns in the form of **triple** (?s for subject, ?p for property and ?o for ?object)

Multiple triples can be provided as graph pattern to match but each triple should end with a period. As an example, let retrieve 5 movies (?movie) along with their titles (?title).

In [0]:
movie_with_title = """
PREFIX vocab: <https://nexus-sandbox.io/v1/vocabs/tutorialnexus/$PROJECTLABEL/>
PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
Select ?movie ?title
 WHERE  {
    ?movie a vocab:Movie.
    ?movie vocab:title ?title.
} LIMIT 5
"""

nexus_results = query_sparql(movie_with_title,sparqlview_wrapper)

nexus_df =sparql2dataframe(nexus_results)
nexus_df.head()

Note PREFIX clauses. It is way to shorten URIS within a SPARQL query. Without them we would have to use full URI for all properties.

The ?movie variable is bound to a URI (the internal Nexus id). Let retrieve the movieId just like in the MovieLens csv files for simplicity.

In [0]:
movie_with_title = """
PREFIX vocab: <https://nexus-sandbox.io/v1/vocabs/tutorialnexus/$PROJECTLABEL/>
PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
Select ?movieId ?title
 WHERE  {
    
    # Select movies
    ?movie a vocab:Movie.

    # Select their movieId value
    ?movie vocab:movieId ?movieId.
    
    #
    ?movie vocab:title ?title.
    
} LIMIT 5
"""

nexus_results = query_sparql(movie_with_title,sparqlview_wrapper)

nexus_df =sparql2dataframe(nexus_results)
nexus_df.head()

In the above query movies are things (or entities) of type vocab:Movie. 
This is a typical instance query where entities are filtered by their type(s) and then some of their properties are retrieved (here ?title). 

Let retrieve everything that is linked (outgoing) to the movies. 
The * character in the SELECT clause indicates to retreve all variables: ?movie, ?p, ?o

In [0]:
movie_with_properties = """
PREFIX vocab: <https://nexus-sandbox.io/v1/vocabs/tutorialnexus/$PROJECTLABEL/>
PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
Select *
 WHERE  {
    ?movie a vocab:Movie.
    ?movie ?p ?o.
} LIMIT 20
"""

nexus_results = query_sparql(movie_with_properties,sparqlview_wrapper)

nexus_df =sparql2dataframe(nexus_results)
nexus_df.head(20)

As a little exercise, write a query retrieving incoming entities to movies. You can copy past the query above and modify it.

Hints: ?s ?p ?o can be read as: ?o is linked to ?s with an outgoing link.

Do you have results ?

In [0]:
#Your query here


Let retrieve the movie ratings

In [0]:
movie_with_properties = """
PREFIX vocab: <https://nexus-sandbox.io/v1/vocabs/tutorialnexus/$PROJECTLABEL/>
PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
Select ?userId ?movieId ?rating ?timestamp
 WHERE  {
    ?movie a vocab:Movie.
    ?movie vocab:movieId ?movieId.
    
    
    ?ratingNode vocab:movieId ?ratingmovieId.
    ?ratingNode vocab:rating ?rating.
    ?ratingNode vocab:userId ?userId.
    ?ratingNode vocab:timestamp ?timestamp.
    
    # Somehow pandas is movieId as double for rating 
    FILTER(xsd:integer(?ratingmovieId) = ?movieId)
    
} LIMIT 20
"""

nexus_results = query_sparql(movie_with_properties,sparqlview_wrapper)

nexus_df =sparql2dataframe(nexus_results)
nexus_df.head(20)

As a little exercise, write a query retrieving the movie tags along with the user id and timestamp. You can copy and past the query above and modify it.


In [0]:
#Your query here



### Aggregate queries

[Aggregates](https://www.w3.org/TR/sparql11-query/#aggregates) apply some operations over a group of solutions.
Available aggregates are: COUNT, SUM, MIN, MAX, AVG, GROUP_CONCAT, and SAMPLE.

We will not see them all but we'll look at some examples.

The next query will compute the average rating score for 'funny' movies and retrieve 5 of them. 

In [24]:
movie_avg_ratings = """
PREFIX vocab: <https://nexus-sandbox.io/v1/vocabs/amld/recommender/>
PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>

Select  DISTINCT ?movieId (AVG(?rating) AS ?score) (count(?rating) AS ?total_ratings)
 WHERE  {
    # Select movies
    ?movie a vocab:Movie.

    # Select their movieId value
    ?movie vocab:movieId ?movieId.

    # Keep movies with 'funny' tags
    ?tagnode vocab:movieId ?movieId.
    ?tagnode vocab:tag "funny".

    # Keep movies with ratings
    ?ratingNode vocab:movieId ?ratingmovieId.
    ?ratingNode vocab:rating ?rating.

    
    FILTER(xsd:integer(?ratingmovieId) = ?movieId)

}
GROUP BY ?movieId
LIMIT 5
"""

nexus_results = query_sparql(movie_avg_ratings,sparqlview_wrapper)

nexus_df =sparql2dataframe(nexus_results)
nexus_df.head(20)



,movieId,score,total_ratings,total_users
0,68848,4.333333333333333,3,0
1,134170,3.5,5,0
2,126548,3.25,6,0
3,167746,4.166666666666667,6,0
4,106766,3.642857142857143,7,0


In [0]:
movie_avg_ratings = """
PREFIX vocab: <https://nexus-sandbox.io/v1/vocabs/amld/recommender/>
PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>

Select  DISTINCT ?movieId (AVG(?rating) AS ?score)
 WHERE  {
    # Select movies
    ?movie a vocab:Movie.

    # Select their movieId value
    ?movie vocab:movieId ?movieId.

    # Keep movies with 'funny' tags
    ?tagnode vocab:movieId ?movieId.
    ?tagnode vocab:tag "funny".

    # Keep movies with ratings
    ?ratingNode vocab:movieId ?movidId.
    ?ratingNode vocab:rating ?rating.

}
GROUP BY ?movieId
LIMIT 5
"""

nexus_results = query_sparql(movie_avg_ratings,sparqlview_wrapper)

nexus_df =sparql2dataframe(nexus_results)
nexus_df.head(20)



The next query will retrieve movies along with users that tagged them separated by a comma

In [12]:
# Group Concat

movie_avg_ratings = """
PREFIX vocab: <https://nexus-sandbox.io/v1/vocabs/amld/recommender/>
PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>

Select ?movieId (group_concat(DISTINCT ?userId;separator=",") as ?users)
 WHERE  {
    # Select movies
    ?movie a vocab:Movie.

    # Select their movieId value
    ?movie vocab:movieId ?movieId.

    ?tagnode vocab:movieId ?movieId.
    ?tagnode vocab:userId ?userId.

  
}
GROUP BY ?movieId
LIMIT 10
"""

nexus_results = query_sparql(movie_avg_ratings,sparqlview_wrapper)

nexus_df =sparql2dataframe(nexus_results)
nexus_df.head(20)

,movieId,users
0,2863,474
1,2078,474
2,2076,474
3,2662,573
4,11,474
5,14,474
6,1643,474
7,1900,474
8,1902,474
9,356,"474,533,567"


In [0]:
movie_avg_ratings = """
PREFIX vocab: <https://nexus-sandbox.io/v1/vocabs/amld/recommender/>
PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>

Select ?movieId (group_concat(DISTINCT ?userId;separator=",") as ?users)
 WHERE  {
    # Select movies
    ?movie a vocab:Movie.

    # Select their movieId value
    ?movie vocab:movieId ?movieId.

    ?tagnode vocab:movieId ?movieId.
    ?tagnode vocab:userId ?userId.

  
}
GROUP BY ?movieId
LIMIT 10
"""

nexus_results = query_sparql(movie_avg_ratings,sparqlview_wrapper)

nexus_df =sparql2dataframe(nexus_results)
nexus_df.head(20)